In [12]:
import os

# Define color mapping for file extensions with desaturated colors
color_map = {
    '.xlsx': 'olive', '.rpt': 'olive', '.csv': 'green',
    '.py': 'steelblue', '.ipynb': 'steelblue',
    '.pdf': 'firebrick', 
    '.ppt': 'saddlebrown', '.pptx': 'saddlebrown',
    '.doc': 'gainsboro', '.docx': 'gainsboro', '.txt': 'gainsboro'
}
directory_color = 'khaki'

# Function to create the directory tree in Markdown format
def create_directory_tree_markdown(start_dir):
    markdown_content = "# Directory Tree\n\n"
    
    for root, dirs, files in os.walk(start_dir):
        # Get depth for indenting in Markdown
        level = root.replace(start_dir, '').count(os.sep)
        indent = '  ' * level
        # Add directory to markdown content with specified color (without trailing "/")
        markdown_content += f"{indent}- <span style='color:{directory_color}'>{os.path.basename(root)}</span>\n"
        
        for file in files:
            file_ext = os.path.splitext(file)[1]
            # Choose color based on file extension
            color = color_map.get(file_ext, 'grey')  # Default color is grey for unspecified extensions
            # Add file with appropriate color
            markdown_content += f"{indent}  - <span style='color:{color}'>{file}</span>\n"
    
    return markdown_content

# Function to save the markdown content to a .md file
def save_to_markdown_file(markdown_content, output_file="directory_tree.md"):
    with open(output_file, 'w') as file:
        file.write(markdown_content)
    print(f"Directory tree saved to {output_file}")

# Example usage:
start_directory = 'dir'  # Set to the starting directory of your choice
markdown_tree = create_directory_tree_markdown(start_directory)
save_to_markdown_file(markdown_tree)


Directory tree saved to directory_tree.md


In [11]:
import os
import tkinter as tk
from tkinter import ttk, messagebox
import webbrowser
import subprocess
import platform

# Define color mapping for file extensions with desaturated colors
color_map = {
    '.xlsx': 'olive', '.rpt': 'olive', '.csv': 'green',
    '.py': 'steelblue', '.ipynb': 'steelblue',
    '.pdf': 'firebrick', 
    '.ppt': 'saddlebrown', '.pptx': 'saddlebrown',
    '.doc': 'gainsboro', '.docx': 'gainsboro', '.txt': 'gainsboro'
}
directory_color = 'khaki'
bg_color = '#2d2d2d'  # Dark background color
text_color = 'white'

# Function to open a file in the default app
def open_file(file_path):
    try:
        if os.path.isdir(file_path):
            # Open the directory in the default file explorer
            if platform.system() == "Windows":
                subprocess.Popen(f'explorer "{file_path}"')
            elif platform.system() == "Darwin":  # macOS
                subprocess.Popen(["open", file_path])
            else:  # Linux and others
                subprocess.Popen(["xdg-open", file_path])
        else:
            webbrowser.open(file_path)  # Open file with the default application
    except Exception as e:
        messagebox.showerror("Error", f"Could not open: {e}")

# Function to create the directory tree structure in the Tkinter Treeview widget
def populate_tree(tree, start_dir, parent="", processed_dirs=set()):
    for root, dirs, files in os.walk(start_dir):
        # Skip if this directory has already been processed
        if root in processed_dirs:
            continue
        processed_dirs.add(root)  # Mark this directory as processed

        root_name = os.path.basename(root) or start_dir
        if root == start_dir:
            node = tree.insert(parent, "end", text=root_name, open=True,
                               values=[root], tags=("directory",))
        else:
            node = tree.insert(parent, "end", text=root_name, open=False,
                               values=[root], tags=("directory",))

        # Insert files with appropriate colors and path values
        for file in files:
            file_ext = os.path.splitext(file)[1]
            file_color = color_map.get(file_ext, 'grey')  # Default color is grey for unspecified extensions
            file_path = os.path.join(root, file)
            tree.insert(node, "end", text=file, values=[file_path], tags=(file_color,))

        # Process directories recursively without re-adding already processed ones
        for directory in dirs:
            dir_path = os.path.join(root, directory)
            if dir_path not in processed_dirs:
                populate_tree(tree, dir_path, node, processed_dirs)

# Function to set up color tags for the Treeview items
def setup_color_tags(tree):
    tree.tag_configure('directory', foreground=directory_color)
    for ext, color in color_map.items():
        tree.tag_configure(color, foreground=color)
    tree.tag_configure('grey', foreground="grey")

# Main application class
class DirectoryTreeApp(tk.Tk):
    def __init__(self, start_dir="dir"):
        super().__init__()
        self.title("Directory Tree Viewer")
        self.geometry("600x400")
        self.configure(bg=bg_color)
        
        # Styling for dark mode
        style = ttk.Style(self)
        style.theme_use("clam")
        style.configure("Treeview", 
                        background=bg_color,
                        foreground=text_color,
                        fieldbackground=bg_color,
                        font=("Arial", 10))
        style.configure("Treeview.Heading", background=bg_color, foreground=text_color)
        
        # Create Treeview widget
        self.tree = ttk.Treeview(self, show="tree")  # Remove extra columns, only show the tree structure
        self.tree.heading("#0", text="Directory Structure")
        self.tree.column("#0", anchor="w")
        
        # Populate the tree and set up color tags
        populate_tree(self.tree, start_dir)
        setup_color_tags(self.tree)
        
        # Add a scrollbar
        y_scroll = ttk.Scrollbar(self, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscroll=y_scroll.set)
        y_scroll.pack(side="right", fill="y")
        
        # Place the Treeview widget
        self.tree.pack(fill="both", expand=True)
        
        # Bind double-click event to open files or directories
        self.tree.bind("<Double-1>", self.on_item_double_click)
        
        # Bind Enter key to open files or directories
        self.tree.bind("<Return>", self.on_item_double_click)  # <-- Added this line

    def on_item_double_click(self, event):
        # Get selected item and open it (file or directory)
        selected_item = self.tree.selection()[0]
        file_path = self.tree.item(selected_item, "values")[0]
        open_file(file_path)


DirectoryTreeApp(start_dir="dir").mainloop()